In [1]:
from IPython.display import display, HTML
import pandas as pd

from consent.data.cookie_setting import CookieSetting
from consent.util.default_path import get_data_dir
from ooutil.df_util import find_one

# pref_btn_dir = get_data_dir("2021-03-01") / 'pref_btn_v7'
# pref_btn_dir = get_data_dir("2021-08-03") / 'pref_btn'
# pref_btn_dirs = [get_data_dir("2021-08-04") / 'pref_btn',  get_data_dir("2021-11-26") / 'pref_btn']
pref_btn_dirs = [get_data_dir("2022-05-24") / 'pref_btn', get_data_dir("2022-05-26") / 'pref_btn']
assert all(pref_btn_dir.exists() for pref_btn_dir in pref_btn_dirs)

In [2]:
# Verify each site
def display_df(df):
    display(HTML(df.to_html()))

def verify_dataset_site(site, df, opt_settings):
    if 'pref_btn' not in df.columns:
        return
    pref_btn_rows = df[df.pref_btn]
    if len(pref_btn_rows) == 0: 
        print(f'{site} pref btn not found')
    if len(pref_btn_rows) != len(opt_settings):
        print('pref_btn_rows')
        display_df(pref_btn_rows)
        print('opt_setting')
        for setting in opt_settings:
            print(setting)
        raise ValueError(f'Unexpected dataset on site {site}')


cookieset = CookieSetting.get_cookie_settings(nocache=True)
pref_data = []
anno_sites = cookieset.site.tolist()
print("Num pref-btn annotated sites:", len(anno_sites))
for site in anno_sites:
        site_files = [f for pref_btn_dir in pref_btn_dirs for f in pref_btn_dir.glob(f'{site}*.csv')] 
        assert len(site_files) > 0, f'{site} files not found'
        dfs = []
        for afile in site_files:
            site = afile.stem.split('_')[0]  # some sites have '_' in name to separate frame
            dfs.append(pd.read_csv(afile, index_col=0))
        df = pd.concat(dfs)
        anno_site = find_one(cookieset, 'site', site)
        assert anno_site is not None, f'{site} invalid anno_site'
        try:
            verify_dataset_site(site, df, anno_site.opt_setting)
        except Exception as e:
            print(f'Error on checking {afile} : {e}')
print('Verification passed.')

Num pref-btn annotated sites: 298
Verification passed.
